In [ ]:
import sys
sys.path.append('../Classes')
from AzureSearch import Config, AzureSearch
import pandas as pd
import openai
from azure.identity import DefaultAzureCredential
from azure.search.documents.indexes.models import (
    VectorSearch,
    HnswVectorSearchAlgorithmConfiguration
)

config = Config(path="../Classes/environment.env")
cred = DefaultAzureCredential()
token = cred.get_token("https://cognitiveservices.azure.com/.default")

openai.api_key = token.token
openai.api_version = config.openai_api_version
openai.api_type = "azure"
openai.api_base = config.openai_api_base

vector_search = VectorSearch(
    algorithm_configurations=[
        HnswVectorSearchAlgorithmConfiguration(
            name=config.vector_config_name,
            kind="hnsw",
            parameters={
                "m": 4,
                "efConstruction": 400,
                "efSearch": 500,
                "metric": "cosine"
            }
        )
    ]
)

def generate_grounded_answer(query, content, system, reminder, config=config):
    # Format the user message using the given query and content
    user_message = f"Source:\n{content}\n Answer the User question: {query}"

    # Construct the message to be sent to the model
    messages = [
        {"role": "system", "content": f"""{system}"""},
        {"role": "user", "content": f"""{user_message}\n{reminder}"""}
    ]

    # Request a response from the model
    response = openai.ChatCompletion.create(
        engine=config.gpt,
        messages=messages,
        temperature=0,
    )
    
    # Extract and return the model's reply
    bot_reply = response['choices'][0]['message']['content']
    return bot_reply

def concatenate_sources(results):
    # Sort the array by '@search.score' in ascending order
    sorted_results = sorted(results, key=lambda x: x['@search.score'])

    # Initialize the 'sources' string
    sources = ""

    # Loop through the sorted results and format the string
    for item in sorted_results:
        title = item.get('title', 'No Title')
        citation_id = item.get('id', 'No ID')
        content = item.get('content', 'No Content')
        sources += f"{title}\nCitation: {citation_id}\n{content}\n\n"

    return sources

def process_queries_and_generate_responses(queries_titles, index_name, k, prompts, config=config):
    azure_search = AzureSearch(config, index_name)
    responses = []  # This list will store the final responses for each query and title
    queries_titles_df = pd.read_csv(queries_titles, encoding='utf-8')
    # Iterate over each row in the queries and titles DataFrame
    for _, row in queries_titles_df.iterrows():
        query = row['query']
        title = row['title']
        filter = f"title eq '{title}'"
        # Use the mock Azure search function to get the top result for the query and title
        search_results = azure_search.vector_search(query, filter, k, "id,title,content")
        # If there are results, get the content of the top result
        content = [i for i in search_results]
        
        # For each prompt in the prompts DataFrame, generate a response using the mock OpenAI function
        prompt_responses = {}
        prompts_df = pd.read_csv(prompts, encoding='utf-8')
        for index, prompt_row in prompts_df.iterrows():
            prompt = prompt_row['prompt']
            reminder = prompt_row['reminder']
            ai_response = generate_grounded_answer(query, content, prompt, reminder, config)
            # Assuming you want the row num to start from 1
            column_name = f'prompt_{index + 1}'  # Adding 1 if the DataFrame index starts at 0
            prompt_responses[column_name] = ai_response

        # Compile the query, title, and all prompt responses into a single record
        record = {
            'query': query,
            'title': title,
            **prompt_responses  # Add the prompt responses with dynamic keys
        }
        responses.append(record)

    # Create a DataFrame from the responses
    final_df = pd.DataFrame(responses)

    return final_df

process_queries_and_generate_responses('./test_csv.txt', config.index_name, config.top_k, './test_csv_prompt.txt')